# MLOps Workshop: Umgebungseinrichtung

## Einführung
Willkommen zum MLOps Workshop! In diesem Notebook richten wir unsere Entwicklungsumgebung ein und installieren alle notwendigen Tools für unser Customer Churn Prediction Projekt.

## Lernziele
Nach Abschluss dieses Notebooks werden Sie:
- Eine funktionierende Python-Umgebung mit allen erforderlichen Paketen haben
- Die grundlegende Projektstruktur verstehen
- Ein Git-Repository initialisiert haben
- Alle MLOps-Tools auf ihre Funktionsfähigkeit überprüft haben

## 1. Umgebungseinrichtung

### 1.1 Virtuelle Umgebung erstellen
Führen Sie zunächst diese Befehle in Ihrem Terminal aus:

```bash
python -m venv mlops-venv
# Unter Windows
.\mlops-venv\Scripts\activate
# Unter Unix/MacOS
source mlops-venv/bin/activate
```

### 1.2 Abhängigkeiten installieren
Installieren wir die benötigten Pakete:

In [13]:
# Zelle 1: Benötigte Pakete installieren
!pip install numpy pandas scikit-learn mlflow pytest fastapi uvicorn great-expectations docker python-dotenv matplotlib seaborn

### 1.3 Installation überprüfen
Führen Sie diese Zelle aus, um zu überprüfen, ob alle Pakete korrekt installiert wurden:

In [15]:
# Zelle 2: Import und Versionscheck
import sys
import numpy as np
import pandas as pd
import mlflow
import great_expectations as ge
from fastapi import FastAPI
import pytest

# Versionen ausgeben
print(f"Python Version: {sys.version}")
print(f"NumPy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"MLflow Version: {mlflow.__version__}")
print(f"Great Expectations Version: {ge.__version__}")

Python Version: 3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
NumPy Version: 1.26.4
Pandas Version: 2.2.3
MLflow Version: 2.19.0
Great Expectations Version: 0.18.22


## 2. Projektstruktur
Unser Projekt wird dieser Struktur folgen:
```
customer_churn/
├── data/
│   ├── raw/
│   └── processed/
├── notebooks/
│   ├── 00_Umgebung_Einrichtung.ipynb
│   ├── 01_Daten_Exploration.ipynb
│   ├── 02_Daten_Vorverarbeitung.ipynb
│   ├── 03_Modell_Engineering.ipynb
│   └── 04_Modell_Deployment.ipynb
├── src/
│   ├── data/
│   ├── features/
│   ├── models/
│   └── api/
├── tests/
├── .gitignore
├── README.md
└── requirements.txt
```

Lassen Sie uns diese Struktur erstellen:

In [3]:
# Zelle 3: Projektstruktur erstellen
import os

def create_project_structure():
    # Verzeichnisstruktur definieren
    directories = [
        'data/raw',
        'data/processed',
        'notebooks',
        'src/data',
        'src/features',
        'src/models',
        'src/api',
        'tests'
    ]
    
    # Verzeichnisse erstellen
    for dir_path in directories:
        os.makedirs(dir_path, exist_ok=True)
        print(f"Verzeichnis erstellt: {dir_path}")

create_project_structure()

Verzeichnis erstellt: data/raw
Verzeichnis erstellt: data/processed
Verzeichnis erstellt: notebooks
Verzeichnis erstellt: src/data
Verzeichnis erstellt: src/features
Verzeichnis erstellt: src/models
Verzeichnis erstellt: src/api
Verzeichnis erstellt: tests


## 3. Git Einrichtung

### 3.1 Git Repository initialisieren
Führen Sie diese Befehle im Terminal aus:

```bash
git init
```

### 3.2 .gitignore erstellen
Erstellen wir eine .gitignore Datei:

In [2]:
# Zelle 4: .gitignore erstellen
gitignore_content = """
# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
env/
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
*.egg-info/
.installed.cfg
*.egg

# Virtuelle Umgebung
mlops-venv/
venv/
ENV/

# Jupyter Notebook
.ipynb_checkpoints

# MLflow
mlruns/

# Daten
data/raw/*
data/processed/*
!data/raw/.gitkeep
!data/processed/.gitkeep

# IDE
.idea/
.vscode/
"""

with open('.gitignore', 'w') as f:
    f.write(gitignore_content)
print(".gitignore Datei wurde erstellt")

.gitignore Datei wurde erstellt


## 4. Datensatz herunterladen
Laden wir den Telco Customer Churn Datensatz herunter:

In [4]:
# Zelle 5: Datensatz herunterladen
import pandas as pd

url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
df = pd.read_csv(url)
df.to_csv('../data/raw/telco_customer_churn.csv', index=False)
print("Datensatz wurde heruntergeladen und in data/raw/telco_customer_churn.csv gespeichert")

Datensatz wurde heruntergeladen und in data/raw/telco_customer_churn.csv gespeichert


In [21]:
# Function to introduce missing values for learning purposes
def introduce_missing_values(df):
    df_missing = df.copy()
    
    # 1. Complete row missing for a few customers (MCAR - Missing Completely At Random)
    random_indices = np.random.choice(df.index, 5, replace=False)
    columns_to_nullify = ['gender', 'Partner', 'Dependents', 'PhoneService', 'InternetService']
    df_missing.loc[random_indices, columns_to_nullify] = np.nan
    
    # 2. Missing values in MonthlyCharges for high tenure customers (MAR - Missing At Random)
    high_tenure_indices = df[df['tenure'] > 60].index
    random_high_tenure = np.random.choice(high_tenure_indices, 10, replace=False)
    df_missing.loc[random_high_tenure, 'MonthlyCharges'] = np.nan
    
    # 3. Missing values in TotalCharges for customers with high MonthlyCharges (MNAR - Missing Not At Random)
    high_charges_indices = df[df['MonthlyCharges'] > 90].index
    random_high_charges = np.random.choice(high_charges_indices, 8, replace=False)
    df_missing.loc[random_high_charges, 'TotalCharges'] = np.nan
    
    # 4. Missing values in streaming services for customers without internet
    no_internet_indices = df[df['InternetService'] == 'No'].index
    random_no_internet = np.random.choice(no_internet_indices, 15, replace=False)
    streaming_columns = ['StreamingTV', 'StreamingMovies']
    df_missing.loc[random_no_internet, streaming_columns] = np.nan

    # 5. Introduce simple duplicates
    # Just duplicate 10 random rows completely
    rows_to_duplicate = df_missing.sample(n=10)
    df_missing = pd.concat([df_missing, rows_to_duplicate], ignore_index=True)
    
    return df_missing

# Load original data
df = pd.read_csv('../data/raw/telco_customer_churn.csv')

# Introduce missing values
df_with_missing = introduce_missing_values(df)

# Save the modified dataset
df_with_missing.to_csv('../data/raw/telco_customer_churn_with_missing.csv', index=False)

## 5. Einrichtung überprüfen
Führen Sie diese abschließende Überprüfung durch:

In [11]:
# Zelle 6: Abschließende Verifizierung
import os

def verify_setup():
    checks = {
        "Projektstruktur": all(os.path.exists(d) for d in ['../data/raw', '../notebooks', '../src', '../tests']),
        "Git initialisiert": os.path.exists('../.git'),
        "Datensatz heruntergeladen": os.path.exists('../data/raw/telco_customer_churn.csv'),
        "Git ignore erstellt": os.path.exists('../.gitignore')
    }
    
    all_passed = True
    for check, passed in checks.items():
        status = "✅" if passed else "❌"
        print(f"{status} {check}")
        all_passed = all_passed and passed
    
    return all_passed

success = verify_setup()
if success:
    print("\nEinrichtung erfolgreich abgeschlossen! Sie können mit dem Workshop beginnen!")
else:
    print("\nEinige Überprüfungen sind fehlgeschlagen. Bitte überprüfen Sie die obige Ausgabe und beheben Sie eventuelle Probleme.")

✅ Projektstruktur
✅ Git initialisiert
✅ Datensatz heruntergeladen
✅ Git ignore erstellt

Einrichtung erfolgreich abgeschlossen! Sie können mit dem Workshop beginnen!


## Nächste Schritte
- Überprüfen Sie die Projektstruktur und die erstellten Dateien
- Erstellen Sie Ihren ersten Git-Commit
- Fahren Sie mit dem Daten-Explorations-Notebook fort

## Übung
1. Initialisieren Sie das Git-Repository
2. Fügen Sie alle Dateien hinzu, außer denen, die in .gitignore erwähnt sind
3. Erstellen Sie Ihren ersten Commit mit der Nachricht "Initial Setup"
4. Erstellen Sie einen neuen Branch namens "development"

### Hilfreiche Git-Befehle für die Übung:
```bash
git init
git add .
git commit -m "Initial Setup"
git branch development
git checkout development
```